In [1]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2
#%matplotlib qt

In [2]:
from skdim import global_id
from skdim import local_id
import skdim
import multiprocessing as mp
import scipy
import numpy as np
import pandas as pd
import rpy2
import rpy2.robjects as ro
import rpy2.robjects.numpy2ri
import rpy2.robjects.packages as rpackages

from sklearn.neighbors import NearestNeighbors
import sklearn.datasets
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

rpy2.robjects.numpy2ri.activate()
utils = rpackages.importr('utils')
#utils.install_packages('intrinsicDimension')
#utils.install_packages('ider')
intdimr = rpackages.importr('intrinsicDimension')
ider   = rpackages.importr('ider')
#r_base = rpackages.importr('base')

import sys
sys.path.append('/home/utilisateur/id-concentration/')
from estimators import *

In [3]:
def compute_betas(max_dim):
    #COMPUTE_BETAS Computes betas between one and max_dim. 
    #   betas = compute_betas( max_dim ) Computes the variance of the angle between two Gaussian
    #   vectors in RR^d, it does so for each d between 1 and max_dim.

    betas = np.zeros(max_dim-2)
    betas[:2] = np.array([np.pi**2/12, np.pi**2/4 - 2])

    if (max_dim >= 4):
        for ii in range(4,max_dim):
            betas[ii-1] = betas[ii-3] - 2/(ii-1)**2

    betas = np.concatenate((np.array([pi**2/4]), betas))

    return betas[:,None]

def compute_cuts(max_dim):
    #COMPUTE_CUTS Computes the cuts that define the thresholds to decide between
    #   etas = compute_cuts(max_dim) Computes the cuts to decide between any
    #   dimension between 1 and max_dim.
    betas = compute_betas(max_dim)
    etas = np.zeros((len(betas)-1,1))
    for ii in range(len(etas)):
        etas[ii] = (betas[ii] + betas[ii+1])/2
    
    return etas

In [5]:
data = np.zeros((30,10))
data[:,:5] = skdim.gendata.hyperBall(n_points = 30, n_dim = 5, radius = 1, random_state = 0)

In [141]:
res2=radovanovic_estimators_matlab(data,k=10)